In [1]:
import pandas as pd
from datetime import datetime
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS, PROV, SKOS, RDFS, FOAF

In [2]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/
PATH = ROOT + "data/ACTIVITIES.csv"
OUTPUT_PATH = ROOT + "data/rdf/"
COMMUNE = "Gentilly"

activities = pd.read_csv(PATH,header=0)
display(activities)

,ID,rdf:type,rdfs:label,rdfs:comment,prov:wasAssociatedWith,foaf:firstName,foaf:familyName,prov:startedAtTime,prov:endedAtTime
0,0001,prov:Activity,Vectorisation manuelle des plans parcellaires,NaN,7da2fd27-7dff-4a00-b3e4-6eb9f972d7e9,NaN,NaN,2024-06-19,2024-06-19
1,0002,prov:Activity,Transcription manuelle des registres,NaN,7da2fd27-7dff-4a00-b3e4-6eb9f972d7e9,NaN,NaN,2024-03-04,2024-03-29
2,0003,prov:Activity,Recallage de plans parcellaires géoréférencés,NaN,7da2fd27-7dff-4a00-b3e4-6eb9f972d7e9,NaN,NaN,2024-04-15,2024-04-15
3,0004,prov:Activity,Géoréférencement des plans du cadastre napoléo...,NaN,c1ca7fba-aa36-45e4-af19-f699ed001e05,NaN,NaN,NaN,2015-03-17
4,0005,prov:Activity,Géoréférencement des plans du cadastre napoléo...,NaN,9222844c-1bfb-4096-bad5-cfa79542ab90,NaN,NaN,NaN,2023-05-16
5,7da2fd27-7dff-4a00-b3e4-6eb9f972d7e9,prov:Agent;foaf:Person,Solenn Tual,NaN,NaN,Solenn,Tual,NaN,NaN
6,a00cf3be-fcd8-47fb-8f62-1107ca771965,prov:SoftwareAgent,DAN,Document Attention Network (proposé par Denis ...,NaN,NaN,NaN,NaN,NaN
7,35e02783-5811-4625-b56b-92e5cb30d7ec,prov:SoftwareAgent,Classifieur,Modèle de deep learning de détection et classi...,NaN,NaN,NaN,NaN,NaN
8,c1ca7fba-aa36-45e4-af19-f699ed001e05,prov:Agent,Service départemental d'Archéologie du Val-de-...,NaN,NaN,NaN,NaN,NaN,NaN
9,9222844c-1bfb-4096-bad5-cfa79542ab90,prov:Agent,Pairs Time Machine,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
vocabs = {
    "prov": PROV,
    "skos": SKOS,
    "rdfs": RDFS,
    "dcterms": DCTERMS,
    "foaf":FOAF
}

classesdict = {
    "prov:Agent":PROV.Agent,
    "prov:SoftwareAgent":PROV.SoftwareAgent,
    "foaf:Person":FOAF.Person,
}

In [4]:
from namespaces import *

g = Graph()
g.bind("activity", activity)

for index, row in activities.iterrows():
    
    if row["rdf:type"] == "prov:Activity":
        activity_ = URIRef(activity + row["ID"])
        g.add((activity_, RDF.type, PROV.Activity))
        g.add((activity_, RDFS.label, Literal(row["rdfs:label"], datatype=XSD.string)))
        agent = BNode()
        g.add((activity_, PROV.wasAssociatedWith, agent))
        for index2, row2 in activities.iterrows():
            if 'Agent' in row2["rdf:type"] and row2["ID"] == row["prov:wasAssociatedWith"]:
                types = row2["rdf:type"].split(';')
                for t in types:
                    vocab, classe = t.split(':')
                    g.add((agent, RDF.type, classesdict[t]))
                    g.add((agent, RDFS.label, Literal(row2["rdfs:label"], datatype=XSD.string)))
                    if not pd.isnull(row2["foaf:firstName"]):
                        g.add((agent, FOAF.firstName, Literal(row2["foaf:firstName"], datatype=XSD.string)))
                    if not pd.isnull(row2["foaf:familyName"]):
                        g.add((agent, FOAF.familyName, Literal(row2["foaf:familyName"], datatype=XSD.string)))
                    if not pd.isnull(row2["rdfs:comment"]):
                        g.add((agent, RDFS.comment, Literal(row2["rdfs:comment"], datatype=XSD.string)))
        
        if not pd.isnull(row["prov:startedAtTime"]):
            g.add((activity_, PROV.startedAtTime, Literal(datetime.strptime(row["prov:startedAtTime"], '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))
        if not pd.isnull(row["prov:endedAtTime"]):
            g.add((activity_, PROV.endedAtTime, Literal(datetime.strptime(row["prov:endedAtTime"], '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))

@prefix activity: <http://data.ign.fr/id/codes/cadastre/activity/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

activity:0001 a prov:Activity ;
    rdfs:label "Vectorisation manuelle des plans parcellaires"^^xsd:string ;
    prov:endedAtTime "2024-06-19T00:00:00"^^xsd:dateTimeStamp ;
    prov:startedAtTime "2024-06-19T00:00:00"^^xsd:dateTimeStamp ;
    prov:wasAssociatedWith [ a prov:Agent,
                foaf:Person ;
            rdfs:label "Solenn Tual"^^xsd:string ;
            foaf:familyName "Tual"^^xsd:string ;
            foaf:firstName "Solenn"^^xsd:string ] .

activity:0002 a prov:Activity ;
    rdfs:label "Transcription manuelle des registres"^^xsd:string ;
    prov:endedAtTime "2024-03-29T00:00:00"^^xsd:dateTimeStamp ;
    prov:startedAtTime "2024-03-04T00:00:00"^^xsd:dateTimeStamp ;
    prov:wasAssociatedWith [ a prov:Ag

<Graph identifier=N68c82695cd104344a76533f9b8bfbf24 (<class 'rdflib.graph.Graph'>)>

In [ ]:
print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_PATH}/activities.ttl", format='turtle')